In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import xgboost as xgb

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import make_scorer,  mean_absolute_error

from scipy.stats.distributions import uniform

# Define useful functions

In [2]:
def getCorrelationMatrix(df, method="pearson"):
    corr = df.corr(method=method)
    
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5},
               annot=True, fmt=".2f")

    
    return corr

# Read data

In [3]:
trainingDataPath = "D:/data/kaggle/PUBG/train.csv"

In [4]:
trainingData = pd.read_csv(trainingDataPath)

# Feature engineering

Kaggle page:
* https://www.kaggle.com/c/pubg-finish-placement-prediction

#### Problem description
In a PUBG game, up to 100 players start in each match (matchId). Players can be on teams (groupId) which get ranked at the end of the game (winPlacePerc) based on how many other teams are still alive when they are eliminated. In game, players can pick up different munitions, revive downed-but-not-out (knocked) teammates, drive vehicles, swim, run, shoot, and experience all of the consequences -- such as falling too far or running themselves over and eliminating themselves.

You are provided with a large number of anonymized PUBG game stats, formatted so that each row contains one player's post-game stats. The data comes from matches of all types: solos, duos, squads, and custom; there is no guarantee of there being 100 players per match, nor at most 4 player per group.

You must create a model which predicts players' finishing placement based on their final stats, on a scale from 1 (first place) to 0 (last place).

### Data
1. train.csv - set for model training and evaluation
1. test.csv - set for testing model predictions

#### Data dictionary
* DBNOs - Number of enemy players knocked.
* assists - Number of enemy players this player damaged that were killed by teammates.
* boosts - Number of boost items used.
* damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
* headshotKills - Number of enemy players killed with headshots.
* heals - Number of healing items used.
* killPlace - Ranking in match of number of enemy players killed.
* killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)
* killStreaks - Max number of enemy players killed in a short amount of time.
* kills - Number of enemy players killed.
* longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
* matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.
* revives - Number of times this player revived teammates.
* rideDistance - Total distance traveled in vehicles measured in meters.
* roadKills - Number of kills while in a vehicle.
* swimDistance - Total distance traveled by swimming measured in meters.
* teamKills - Number of times this player killed a teammate.
* vehicleDestroys - Number of vehicles destroyed.
* walkDistance - Total distance traveled on foot measured in meters.
* weaponsAcquired - Number of weapons picked up.
* winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)
* groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
* numGroups - Number of groups we have data for in the match.
* maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
* winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

In [5]:
trainingData.dtypes

Id                   int64
groupId              int64
matchId              int64
assists              int64
boosts               int64
damageDealt        float64
DBNOs                int64
headshotKills        int64
heals                int64
killPlace            int64
killPoints           int64
kills                int64
killStreaks          int64
longestKill        float64
maxPlace             int64
numGroups            int64
revives              int64
rideDistance       float64
roadKills            int64
swimDistance       float64
teamKills            int64
vehicleDestroys      int64
walkDistance       float64
weaponsAcquired      int64
winPoints            int64
winPlacePerc       float64
dtype: object

In [6]:
trainingData.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,0,24,0,0,5,247.30,2,0,4,17,...,1,591.3,0,0.0,0,0,782.40,4,1458,0.8571
1,1,440875,1,1,0,37.65,1,1,0,45,...,0,0.0,0,0.0,0,0,119.60,3,1511,0.0400
2,2,878242,2,0,1,93.73,1,0,2,54,...,1,0.0,0,0.0,0,0,3248.00,5,1583,0.7407
3,3,1319841,3,0,0,95.88,0,0,0,86,...,0,0.0,0,0.0,0,0,21.49,1,1489,0.1146
4,4,1757883,4,0,1,0.00,0,0,1,58,...,0,0.0,0,0.0,0,0,640.80,4,1475,0.5217


In [7]:
trainingData.describe()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,...,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06,4.357336e+06
mean,3.102678e+06,1.024777e+06,2.385539e+04,2.656320e-01,9.636856e-01,1.329033e+02,6.901455e-01,2.385866e-01,1.187169e+00,4.703440e+01,...,1.649345e-01,4.238835e+02,2.557985e-03,4.136261e+00,1.388555e-02,5.146264e-03,1.055122e+03,3.457289e+00,1.500504e+03,4.718663e-01
std,1.797477e+06,6.967197e+05,1.378227e+04,6.342160e-01,1.560643e+00,1.699439e+02,1.191514e+00,6.103033e-01,2.366389e+00,2.732772e+01,...,4.672004e-01,1.222927e+03,6.346679e-02,2.757015e+01,1.329266e-01,7.425362e-02,1.116122e+03,2.402109e+00,4.253571e+01,3.079147e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500000e+02,0.000000e+00
25%,1.537746e+06,4.741378e+05,1.191400e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.300000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.334000e+02,2.000000e+00,1.491000e+03,1.979000e-01
50%,3.110606e+06,9.430320e+05,2.383700e+04,0.000000e+00,0.000000e+00,8.776000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.732000e+02,3.000000e+00,1.500000e+03,4.583000e-01
75%,4.657295e+06,1.418544e+06,3.580100e+04,0.000000e+00,1.000000e+00,1.884000e+02,1.000000e+00,0.000000e+00,1.000000e+00,7.000000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.808000e+03,5.000000e+00,1.510000e+03,7.407000e-01
max,6.224522e+06,2.700605e+06,4.773300e+04,2.000000e+01,1.800000e+01,6.384000e+03,6.300000e+01,2.600000e+01,5.900000e+01,1.000000e+02,...,4.100000e+01,4.839000e+04,4.200000e+01,5.286000e+03,6.000000e+00,5.000000e+00,1.730000e+04,7.600000e+01,1.923000e+03,1.000000e+00


In [8]:
sampleTrainingData = trainingData[:300000]

In [9]:
sampleTrainingData.describe()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,300000.000000,3.000000e+05,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,...,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,204389.025837,1.037695e+06,23092.860627,0.332613,1.081897,158.625853,0.830127,0.295047,1.331657,43.444277,...,0.186507,471.597186,0.003050,4.278871,0.013797,0.005497,1121.249752,3.685047,1504.713457,0.495785
std,123510.767527,7.190835e+05,13949.010083,0.760309,1.656428,205.733674,1.445004,0.717985,2.522066,27.664437,...,0.502118,1311.253915,0.062188,27.778589,0.132463,0.076767,1146.533271,2.865617,39.148813,0.312970
min,0.000000,2.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,385.000000,0.000000
25%,95457.750000,4.648562e+05,10714.750000,0.000000,0.000000,19.350000,0.000000,0.000000,0.000000,19.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,149.300000,2.000000,1493.000000,0.215100
50%,211371.500000,9.479690e+05,22733.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,41.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,661.400000,3.000000,1500.000000,0.489100
75%,306828.250000,1.437213e+06,35233.250000,0.000000,2.000000,213.700000,1.000000,0.000000,2.000000,67.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1925.000000,5.000000,1514.000000,0.777800
max,422742.000000,2.700605e+06,47733.000000,16.000000,15.000000,4981.000000,34.000000,20.000000,55.000000,100.000000,...,18.000000,48390.000000,6.000000,1956.000000,6.000000,3.000000,13250.000000,67.000000,1917.000000,1.000000


# Ideas

When do I get beat down in games?
* When everyone else is better
* When my group is inexperienced
* When my team is not well balanced
* When my team is smaller than the competition
* playing an unfamiliar character, weapon, location

When do I win?
* When we have a strong combo of ultimate skills
* when a new feature or character is released and it is overpowered
* When everyone is clear on the optimal strategy

How to operationalize these?
* Group level
    * user level
        * scale and center features
    * group level aggregation of all features
    * match level aggregation of all features

In [24]:
def engineerFeatures(data):
    
    engineered = data.copy()
    
    ###########################
    # User level features
    
    ########################### 
    # Group level features
    # number of players in group
    # mean score of gropu
    groupLevelFeatures = (
        engineered.groupby(["matchId", "groupId"]).agg(
            {
                "Id": "count",
                "winPoints": "mean"
            }
        )
        .reset_index()
    )
    groupLevelFeatures.rename(
        columns={
            "Id": "numPlayersInGroup",
            "winPoints": "meanGroupWinPoints"
        }, 
        inplace=True
    )
    
    engineered = pd.merge(
        engineered,
        groupLevelFeatures,
        on=["matchId", "groupId"],
        how="left"
        
    )
    
    ###########################
    # Match level features
    
    matchLevelFeatures = (
        engineered.groupby(["matchId"]).agg(
            {
                "Id": "count",
                "winPoints": "mean"
            }
        )
        .reset_index()
    )
    matchLevelFeatures.rename(
        columns={
            "Id": "numPlayersInMatch",
            "winPoints": "meanMatchWinPoints"
        }, 
        inplace=True
    )
    
    engineered = pd.merge(
        engineered,
        matchLevelFeatures,
        on=["matchId"],
        how="left"
        
    )
    
    return engineered

In [25]:
engineered = engineerFeatures(sampleTrainingData)

In [26]:
engineered.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,meanGroupWinPoints,numPlayersInGroup,meanMatchWinPoints,numPlayersInMatch
0,0,24,0,0,5,247.30,2,0,4,17,...,0,0,782.40,4,1458,0.8571,1457.500000,2,1479.000000,7
1,1,440875,1,1,0,37.65,1,1,0,45,...,0,0,119.60,3,1511,0.0400,1511.000000,1,1543.857143,7
2,2,878242,2,0,1,93.73,1,0,2,54,...,0,0,3248.00,5,1583,0.7407,1574.333333,3,1519.857143,7
3,3,1319841,3,0,0,95.88,0,0,0,86,...,0,0,21.49,1,1489,0.1146,1489.000000,1,1494.857143,7
4,4,1757883,4,0,1,0.00,0,0,1,58,...,0,0,640.80,4,1475,0.5217,1493.500000,2,1507.714286,7
